# DLinear 预测可视化

这个 notebook 用于对比 `forecast_results.csv` 中的预测值与真实值（`y`）。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')

In [ ]:
# 读取数据
hist = pd.read_csv('data/data_cleaned.csv', parse_dates=['ds'])
cv = pd.read_csv('forecast_results.csv', parse_dates=['ds', 'cutoff'])

# 自动识别模型预测列（排除通用字段）
base_cols = {'unique_id', 'ds', 'cutoff', 'y'}
pred_cols = [c for c in cv.columns if c not in base_cols]
if not pred_cols:
    raise ValueError('forecast_results.csv 中未找到预测列。')
pred_col = pred_cols[0]
print('预测列:', pred_col)

# 安全处理：需求预测通常应非负
cv[pred_col] = cv[pred_col].clip(lower=0)

print('history rows:', len(hist), 'series:', hist['unique_id'].nunique())
print('cv rows:', len(cv), 'cutoff count:', cv['cutoff'].nunique())

In [ ]:
# 选择一个 cutoff（默认选最新）
selected_cutoff = cv['cutoff'].max()
cv_cut = cv[cv['cutoff'] == selected_cutoff].copy()
print('selected cutoff:', selected_cutoff.date(), 'rows:', len(cv_cut))

# 选择需要画图的 SKU：按预测期真实销量排序，取前 N 个
top_n = 9
top_ids = (
    cv_cut.groupby('unique_id')['y']
    .sum()
    .sort_values(ascending=False)
    .head(top_n)
    .index
)
top_ids[:5]

In [ ]:
# 单 SKU 可视化函数：历史 + 真实未来 + 预测未来
def plot_one(uid, history_months=24):
    h = hist[hist['unique_id'] == uid].sort_values('ds').copy()
    f = cv_cut[cv_cut['unique_id'] == uid].sort_values('ds').copy()
    if f.empty:
        return

    cutoff = f['cutoff'].iloc[0]
    hist_tail = h[h['ds'] <= cutoff].tail(history_months)

    plt.figure(figsize=(10, 4))
    plt.plot(hist_tail['ds'], hist_tail['y'], label='History', marker='o', linewidth=1.5)
    plt.plot(f['ds'], f['y'], label='Actual', marker='o', linewidth=2)
    plt.plot(f['ds'], f[pred_col], label='Forecast', marker='o', linewidth=2)
    plt.axvline(cutoff, color='gray', linestyle='--', alpha=0.8, label='Cutoff')
    plt.title(f'{uid} | cutoff={cutoff.date()}')
    plt.xlabel('Month')
    plt.ylabel('Demand')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# 批量画 top SKU
for uid in top_ids:
    plot_one(uid, history_months=24)

In [ ]:
# 聚合层（全量）对比：真实 vs 预测
agg = cv_cut.groupby('ds', as_index=False)[['y', pred_col]].sum()

plt.figure(figsize=(10, 4))
plt.plot(agg['ds'], agg['y'], marker='o', linewidth=2, label='Actual (Total)')
plt.plot(agg['ds'], agg[pred_col], marker='o', linewidth=2, label='Forecast (Total)')
plt.title(f'Aggregate Forecast vs Actual | cutoff={selected_cutoff.date()}')
plt.xlabel('Month')
plt.ylabel('Total Demand')
plt.legend()
plt.tight_layout()
plt.show()

display(agg)

In [ ]:
# 基础误差指标（选定 cutoff）
eps = 1e-9
err = (cv_cut['y'] - cv_cut[pred_col]).abs()
mae = err.mean()
rmse = np.sqrt(((cv_cut['y'] - cv_cut[pred_col]) ** 2).mean())
wape = err.sum() / (cv_cut['y'].abs().sum() + eps)
smape = (2 * err / (cv_cut['y'].abs() + cv_cut[pred_col].abs() + eps)).mean()

pd.DataFrame({
    'metric': ['MAE', 'RMSE', 'WAPE', 'sMAPE'],
    'value': [mae, rmse, wape, smape]
})